<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Read-the-data" data-toc-modified-id="Read-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Read the data</a></span></li><li><span><a href="#Quick-EDA" data-toc-modified-id="Quick-EDA-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Quick EDA</a></span></li><li><span><a href="#Use-PCA-to-reduce-to-2D" data-toc-modified-id="Use-PCA-to-reduce-to-2D-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Use PCA to reduce to 2D</a></span><ul class="toc-item"><li><span><a href="#Evaluate-the-model" data-toc-modified-id="Evaluate-the-model-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Evaluate the model</a></span></li><li><span><a href="#Create-the-biplot" data-toc-modified-id="Create-the-biplot-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Create the biplot</a></span></li></ul></li></ul></div>

# PCA Plotly Biplot

This notebook will implement a toy example of PCA using a standard ML datasets (i.e. Iris Flowers) and draw the biplot chart. This notebook was inspired by the [plotly PCA demonstration](https://plot.ly/python/v3/ipython-notebooks/principal-component-analysis/) and the well-known book Introduction to Statistical Learning (ISLR) which can be found [here](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf).

## Introduction

PCA or Principal Component analysis (without going into any mathematical terminology) is an unsupervised method that let's you reduce a dataset with multiple dimensions into a much smaller dimensional space (usually 2). Some common use cases of this method include :

- Be able to visualise a high dimensional dataset in 2D
- Reducing computational speed by reducing the dimensions prior to feeding the data to a model

**Biplot**

A biplot is a scatter plot that displays all data points in the reduced 2D space along with vector lines that display the loading of each original feature (put simply, "how much" of each principal component is composed of the original feature). 

This is best illustrated with an example taken from page 378 from ISLR. It displays each data point, in this case a US state, reduced to two principal components. The original dataset contains information with four features namely, population (UrbanPop) and crime stats from Rape, Assault and Murder. The aim of this notebook is to produce the same chart using the Iris Dataset in Python. 

Anyone reading this will hopefully see the benefit of these charts but also the power of interractive plotting using plotly!

![alt text](../pca_plot.PNG)

## Libraries

In [6]:
# data manipulation libraries
import pandas as pd
import numpy as np

# visualisation
import plotly.express as px

# Offline mode
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# pre-processing and modelling
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Read the data

For this notebook we are using the Iris dataset but the functions should be applicable to any PCA examples. The data was downloaded from [here](https://www.kaggle.com/saurabh00007/iriscsv/data). We can see from the preview that we are dealing with a fairly small dataset with 4 features:

- Sepal length
- Sepal width
- Petal length
- Petal width

In [7]:
# read the data
df_iris = pd.read_csv("../data/iris.csv")

# simplified names
new_cols = ['id', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

# rename columns
df_iris.columns = new_cols

# print a preview
df_iris.head(10)

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa
7,8,5.0,3.4,1.5,0.2,Iris-setosa
8,9,4.4,2.9,1.4,0.2,Iris-setosa
9,10,4.9,3.1,1.5,0.1,Iris-setosa


In [8]:
df_iris.describe(include="all")

,id,sepal_length,sepal_width,petal_length,petal_width,species
count,150.000000,150.000000,150.000000,150.000000,150.000000,150
unique,NaN,NaN,NaN,NaN,NaN,3
top,NaN,NaN,NaN,NaN,NaN,Iris-virginica
freq,NaN,NaN,NaN,NaN,NaN,50
mean,75.500000,5.843333,3.054000,3.758667,1.198667,NaN
std,43.445368,0.828066,0.433594,1.764420,0.763161,NaN
min,1.000000,4.300000,2.000000,1.000000,0.100000,NaN
25%,38.250000,5.100000,2.800000,1.600000,0.300000,NaN
50%,75.500000,5.800000,3.000000,4.350000,1.300000,NaN
75%,112.750000,6.400000,3.300000,5.100000,1.800000,NaN


## Quick EDA

Quick exploration using plotly to demonstrate the effect of each feature on the species.

In [9]:
# loop through all features
features = df_iris.drop(['id', 'species'], axis=1).columns

for feature in features:
    
    # create a histogram using that feature
    fig = px.histogram(df_iris, x=feature,
                       color="species",
                       nbins=40,
                       template="plotly_white",
                       marginal='box',
                       color_discrete_sequence=['#FF428B', '#425BFF', '#FFD126'])

    # update the bars of the histogram
    fig.update_traces(marker_line_width=.5,
                      marker_line_color="black",
                      opacity=0.7)

    
    # update the layout of the figure
    fig.update_layout(
        title_text=f"Iris Dataset : {feature} distribution by species",
        titlefont_size= 14,
        xaxis_title_text=f'{feature}',
        xaxis_tickfont_size = 10,
        xaxis_title_font_size = 10,
        yaxis_title_text='Count',
        yaxis_tickfont_size = 10,
        yaxis_title_font_size = 10,
        barmode='overlay',
        bargap=0.1    
    )

#     fig.show(renderer = "notebook_connected")
    iplot(fig)

## Use PCA to reduce to 2D

The code below reduces the Iris dataset from 4 dimensions into its 2 principal components using the PCA model from scikit-learn.

In [10]:
components_to_use = 2
target_col = "species"
df_model = df_iris.copy()

# get the feature values
X = df_model[features].values

# apply standardisation
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

# create the class using 2 components
model_pca = PCA(n_components=components_to_use)

# apply the transformation
pca_results = model_pca.fit_transform(X_std)

# create a dataframe with the results
df_results = pd.DataFrame(pca_results, columns=[f'PC{x}' for x in range(1,components_to_use + 1)])

# add the species to the results
df_results[target_col] = df_model[target_col]

df_results.head()

,PC1,PC2,species
0,-2.264542,0.505704,Iris-setosa
1,-2.086426,-0.655405,Iris-setosa
2,-2.367950,-0.318477,Iris-setosa
3,-2.304197,-0.575368,Iris-setosa
4,-2.388777,0.674767,Iris-setosa


### Evaluate the model

PCA has various ways to be evaluated but the two things we will look at are:
    
- Variance explain ratio : This gives you an idea of how much variance you can explain using the reduced dimensions. A simple way I want to think about this, is how much information was lost by carrying out the dimensionality reduction.


- Components: These are the loadings of each original feature on the two components.

A good explanation how PCA works and what these components are can be found [in this video.](https://www.youtube.com/watch?v=FgakZw6K1QQ)

In [11]:
# Print the explained variance ratios
ratios = model_pca.explained_variance_ratio_

for i, ratio in enumerate(ratios):
    
    print(f"Principal component {i+1} explained variance: {ratio.round(4)}")

Principal component 1 explained variance: 0.7277
Principal component 2 explained variance: 0.2303


In [12]:
loadings = model_pca.components_.round(4)

# get the components
for n, feature in enumerate(features):
    
    # get the loading
    
    print(f"For feature: {feature}")
    print(f"Principal Component 1 loading: {loadings[:, n][0]}")
    print(f"Principal Component 2 loading: {loadings[:, n][1]}")
    print()

For feature: sepal_length
Principal Component 1 loading: 0.5224
Principal Component 2 loading: 0.3723

For feature: sepal_width
Principal Component 1 loading: -0.2634
Principal Component 2 loading: 0.9256

For feature: petal_length
Principal Component 1 loading: 0.5813
Principal Component 2 loading: 0.0211

For feature: petal_width
Principal Component 1 loading: 0.5656
Principal Component 2 loading: 0.0654



From the above we can see that petal_width has almost no loading in principal component 2 but a relatively high loading on component 1. This suggests that for samples with high petal_width we expect them to have a high positive PC1 value. If the same sample also has high sepal_width we expect it to have a high positive value of PC2 (high loading). This is demonstrated better below.

### Create the biplot

The biplot below displays the Iris dataset reduced to 2 principal components. The first thing we notice is the immediate separation between Iris-setosa and the rest of the species. The white arrows display the loadings of each original feature. 

We can see that **"petal_width"** and **"petal_length"** have a high positive value of principal component 1 but a small value of principal component 2. The fact that all Iris-setosa samples are at the far negative side of principal component 1 suggests that they must have much lower values of both **"petal_width"** and **"petal_length"** and this is what makes the separate from the other species. This can be confirmed by looking at the last two histograms on the EDA section. Similar observations can be made for the other features.

Of course this is a standard ML example with nice separation and in real life it is unlikely to get datasets that are so easy to work with. Never the less this graph demonstrates how you can compress a lot of information into one graph by using the power of PCA and plotly.

In [13]:
# build the chart
fig = px.scatter(df_results.round(2),
                 x="PC1",
                 y="PC2",
                 color=target_col,
                 template="plotly_dark",
                 color_discrete_sequence=['#FF428B', '#425BFF', '#FFD126'])

# update the markers
fig.update_traces(
#                   marker_line_width = 2,
                  marker_size=8,
                  opacity=0.5)

fig.update_xaxes(dtick=0.5,
                 range=[-3,3],
                 title_text='Principal Component 1',
                 title_font_size = 10,
                 tickfont_size = 10,
                 zeroline=False,
                 zerolinewidth=0,
                 zerolinecolor='lightgrey',
                 showgrid=True)

fig.update_yaxes(dtick=0.5,
                 title_text='Principal Component 2',
                 title_font_size = 10,
                 tickfont_size = 10,
                 zeroline=False,
                 zerolinewidth=0,
                 zerolinecolor='lightgrey',
                 showgrid=True)


annots = []

for i, feature in enumerate(features):
    
    pc1_val = model_pca.components_[:, i][0]
    pc2_val = model_pca.components_[:, i][1]
    
    # add the arrow annotation
    annots.append(dict(
                        x=pc1_val*2.5,
                        y=pc2_val*2.5,
                        xref="x",
                        yref="y",
                        axref="x",
                        ayref="y",
                        arrowwidth=1,
                        showarrow=True,
                        ax=0,
                        ay=0,
                        text="",
                        arrowhead = 2
                        )),
    # add the text annotation
    annots.append(
                    dict(
                        x=pc1_val*2.5 + 0.4,
                        y=pc2_val*2.5 + 0.0,
                        xref="x",
                        yref="y",
                        showarrow=False,
                        font_size=9,
                        text=f"{feature}",
                        )
                )





# # update the layout of the figure
fig.update_layout(
        title_text=f"PCA results with 2 Components",
        titlefont_size= 14,
        barmode='overlay',
        bargap=0.1,   
        annotations=annots
    )

# fig.show(renderer = "notebook_connected")
iplot(fig)

Last updated date: 16/03/2020

This notebook is part of a wider fun project I am carrying out on creating "out-of-the-box" visuals plotted in Matplotlib / Seaborn or even other languages such as R using the interactive library Plotly. You can find more about this project on [my GitHub page](https://github.com/Stratoshad/Plotly_projects). Please feel free to contribute / provide feedback on this project.